In [1]:
import torch
import transformers
from transformers import AutoModel, AutoTokenizer, pipeline, AutoModelForSequenceClassification
from dotenv import load_dotenv
import pandas as pd
import os
from dotenv import load_dotenv
import time
import re
import random

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
huggingface_token = os.getenv("huggingface_token")

In [38]:
root_dir = '/'.join(os.getcwd().split('/')[:-2])
dummy_text = pd.read_csv(os.path.join(root_dir, 'src', 'data', 'dummy_articles', 'articles.csv'))

In [4]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-13b-chat-hf', token=huggingface_token)

pipeline = transformers.pipeline(
    'text-generation',
    model='meta-llama/Llama-2-13b-chat-hf',
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device='cuda:0'
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.42s/it]


In [29]:
# 사용 예시
role_prompts = [
    "You are a lawyer working for an automotive company. In this role, you are responsible for reviewing articles related to the automotive industry.",
    "You are a legal consultant specialized in the automotive sector. Your job includes evaluating the relevance of articles pertaining to the car industry.",
    "You serve as in-house counsel for a company in the automobile manufacturing sector. Part of your duties includes reviewing written content that might be related to automotive matters.",
    "As an attorney working for a corporation involved in automotive technologies, you are tasked with determining the applicability of various articles to the automotive field."
]

step_prompts = [
    "Step 1 - Understand the text separated by three backticks. Step 2 - As a lawyer, determine whether the text is related to the automotive domain. Step 3 - If the text is related to the automotive domain, respond with 'True'; otherwise, respond with 'False.'"
]

example_prompts = [
    "For example: <Article>: '''Article about the latest advancements in electric vehicle technology''' <Laywer>: True",
    "For example: <Article>: '''Legal analysis of environmental regulations impacting automobile manufacturers''' <Lawyer>: False",
    "For example: <Article>: '''Recipe for a delicious cake''' <Lawyer>: False",
    "For example: <Article>: '''Comparing different types of smartphone processors''' <Lawyer>: False"
]

last_prompts = [
    "Please determine whether this text is related to the automotive domain or not. <Lawyer>: ",
    "Review the text and determine if it is related to the automotive domain and respond True or False. <Lawyer>: ",
    "Please review the text and determine and respond True if it is related to the automotive domain. if not related respond False. <Lawyer>: ",
    "As a legal expert specializing in the automotive sector, your task is to review the text enclosed by three backticks and determine its relevance to the automotive industry. If the text is relevant, please respond with 'True.' If it is not, respond with 'False. <Lawyer>: '"
]

In [40]:
def answer(input_text):
    sequences = pipeline(
                    input_text,
                    do_sample=True,
                    top_k=10,
                    num_return_sequences=1,
                    eos_token_id=tokenizer.eos_token_id,
                    max_length=4096,
                    temperature=0.95,
                    repetition_penalty=1.7,
                    no_repeat_ngram_size=3,
                    top_p=0.92
            )
    
    return sequences[0]['generated_text'].replace(input_text, "")

In [41]:
def cleaning_text(text):
    text = text.strip()  # 문장 앞뒤의 공백 제거
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)  # 알파벳과 띄어쓰기를 제외한 모든 문자 제거
    return cleaned_text

In [42]:
def generate_input_text(role_prompts, step_prompts, example_prompts, last_prompts, article):
    """
    Generate input text by randomly choosing a prompt from each category.
    
    Parameters:
    role_prompts (list): List of role prompts
    step_prompts (list): List of step prompts
    example_prompts (list): List of example prompts
    last_prompts (list): List of last prompts
    article (str): The article text
    
    Returns:
    str: Generated input text
    """
    chosen_role_prompt = random.choice(role_prompts)
    chosen_step_prompt = random.choice(step_prompts)
    chosen_example_prompt = random.choice(example_prompts)
    chosen_last_prompt = random.choice(last_prompts)
    
    input_text = (chosen_role_prompt + '\n' + 
                  chosen_step_prompt + '\n' + 
                  chosen_example_prompt + '\n' + 
                  f"'''{article}'''" + '\n' + 
                  chosen_last_prompt)
    
    return input_text

In [43]:
result_dict = {}
cleaned_output_dict = {}

max_tries = 50
time_limit = 60*3


for index, row in dummy_text.iterrows():
    article = cleaned_text = re.sub(r'[^\w\s]', '', row.Content).strip()
    start_time = time.time()
    
    for i in range(max_tries):
        input_text = generate_input_text(role_prompts, step_prompts, [example_prompts[0]], last_prompts, article)
        gen_text = answer(input_text)
        cleaned_output = cleaning_text(gen_text)

        elapsed_time = time.time() - start_time

        if "True" in cleaned_output and "False" not in cleaned_output:
            output = True
            break
        elif "False" in cleaned_output and "True" not in cleaned_output:
            output = False
            break
        elif elapsed_time > time_limit:
            output = None
            break
            
    else:  # max_tries까지 돌고도 원하는 값을 얻지 못했을 경우
        output = None
        
    result_dict[row.Category] = output
    cleaned_output_dict[row.Category] = cleaned_output

/usr/local/lib/python3.9/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [44]:
result_dict

{'Travel': False,
 'Technology': None,
 'Science': False,
 'Food and Cooking': False,
 'Literature': False,
 'Personal Development': False,
 'Entertainment': False,
 'Environmental Issues': False,
 'Art and Creativity': False,
 'Automotive': None}

In [45]:
result_dict = {}
cleaned_output_dict = {}
time_limit = 60*1

for index, row in dummy_text.iterrows():
    article = cleaned_text = re.sub(r'[^\w\s]', '', row.Content).strip()
    start_time = time.time()

    elapsed_time = 0
    while elapsed_time < time_limit:
        input_text = generate_input_text(role_prompts, step_prompts, [example_prompts[0]], last_prompts, article)
        gen_text = answer(input_text)
        cleaned_output = cleaning_text(gen_text)

        if "True" in cleaned_output and "False" not in cleaned_output:
            output = True
            break
        elif "False" in cleaned_output and "True" not in cleaned_output:
            output = False
            break

        elapsed_time = time.time() - start_time
            
    else:  # max_tries까지 돌고도 원하는 값을 얻지 못했을 경우
        output = None

    result_dict[row.Category] = output
    cleaned_output_dict[row.Category] = cleaned_output

In [46]:
result_dict

{'Travel': False,
 'Technology': False,
 'Science': False,
 'Food and Cooking': False,
 'Literature': False,
 'Personal Development': False,
 'Entertainment': False,
 'Environmental Issues': False,
 'Art and Creativity': False,
 'Automotive': False}

In [47]:
cleaned_output_dict

{'Travel': 'False',
 'Technology': 'False',
 'Science': 'False',
 'Food and Cooking': 'False',
 'Literature': 'Falselaw',
 'Personal Development': 'Please examine the provided content below within tildes regarding GoalSetting from wwwyourchangenow com False',
 'Entertainment': 'False',
 'Environmental Issues': 'False',
 'Art and Creativity': 'False',
 'Automotive': 'False'}